# IMSB-2025 Experiment Notebook

**Paper ID:** IMSB-2025  
**Paper Title:** Identifying and Mitigating Social Bias Knowledge in Language Models  

### Goal

Replicate the knowledge-localization approach using triplet-based probes:
- Load triplets (Subject, Relation, Object).
- Probe the model to see how it completes sentences derived from triplets.
- Identify biased associations stored in parameters (simulated via output analysis).


### Experiment Metadata

In [1]:
PAPER_ID = "IMSB-2025"
PAPER_TITLE = "Identifying and Mitigating Social Bias Knowledge in Language Models"

MODEL_NAME = "Salesforce/codegen-350M-mono"
MODEL_TAG = "codegen350M"

DOMAIN = "Social Bias Knowledge / Triplet Probes"

SENSITIVE_ATTRS = ["gender", "race", "religion"]

MAX_NEW_TOKENS = 100
TEMPERATURE = 0.5
DO_SAMPLE = True

### Imports and Environment Check

In [2]:
import os
import json
from datetime import datetime
from pathlib import Path
from transformers import pipeline, set_seed

def check_pkg(name):
    try:
        __import__(name)
        return True
    except Exception as e:
        return f"Missing or error: {e}"

checks = {
    "torch": check_pkg("torch"),
    "transformers": check_pkg("transformers"),
}

checks

{'torch': True, 'transformers': True}

### Create Experiment Folders and Initialize Log

In [3]:
import csv
PHASE = "baseline"
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_ID = f"{PAPER_ID}_{MODEL_TAG}_{PHASE}_{TIMESTAMP}"

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PROMPT_DIR = ROOT / "prompts" / PAPER_ID
PAPER_OUTPUT_ROOT = ROOT / "outputs" / PAPER_ID
RUN_DIR = PAPER_OUTPUT_ROOT / PHASE / "runs" / RUN_ID

GEN_DIR = RUN_DIR / "generated"
AST_DIR = RUN_DIR / "ast_extract"
TEST_DIR = RUN_DIR / "tests_generated"
METRICS_DIR = PAPER_OUTPUT_ROOT / "metrics"
MANIFEST_DIR = PAPER_OUTPUT_ROOT / "manifests"
NOTES_DIR = ROOT / "notes"

for d in [GEN_DIR, AST_DIR, TEST_DIR, METRICS_DIR, MANIFEST_DIR, NOTES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def update_manifests(run_id, paper_id, phase, model, timestamp, gen_path, metrics_path):
    entry = {
        "Run_ID": run_id, "Paper_ID": paper_id, "Phase": phase, 
        "Model": model, "Timestamp": timestamp, 
        "Generated_Path": str(gen_path), "Metrics_Path": str(metrics_path)
    }
    p_manifest = MANIFEST_DIR / "run_manifest.csv"
    write_header = not p_manifest.exists()
    with open(p_manifest, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=entry.keys())
        if write_header: writer.writeheader()
        writer.writerow(entry)
    g_manifest = ROOT / "outputs" / "run_manifest_all.csv"
    with open(g_manifest, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=entry.keys())
        writer.writerow(entry)

LOG_PATH = NOTES_DIR / f"{PAPER_ID}_notes.md"
print(f"Run ID: {RUN_ID}")
print(f"Experiment structure ready for {PAPER_ID}")


Run ID: IMSB-2025_codegen350M_baseline_20260219_175659
Experiment structure ready for IMSB-2025


### Load Knowledge Triplets

In [4]:
PROMPT_FILE = PROMPT_DIR / f"{PAPER_ID}_prompts.json"
with open(PROMPT_FILE, "r") as f:
    data = json.load(f)

triplets = data["triplets"]
template = data["templates"][0]
print(f"Loaded {len(triplets)} triplets and base template.")

Loaded 2 triplets and base template.


### Load Model Pipeline

In [5]:
generator = pipeline("text-generation", model=MODEL_NAME, device=-1)
print("Model loaded successfully.")

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Model loaded successfully.


### Run Triplet Discovery Probes

In [6]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

for t in triplets:
    prompt_text = template.replace("[SUBJECT]", t["subject"]).replace("[RELATION]", t["relation"]).replace("[OBJECT]", t["object"])
    out = generator(prompt_text, max_new_tokens=MAX_NEW_TOKENS, temperature=TEMPERATURE, do_sample=DO_SAMPLE)
    generated_text = out[0]["generated_text"]
    
    out_file = GEN_DIR / f"{PAPER_ID}_{MODEL_TAG}_{t['id']}_{timestamp}.txt"
    with open(out_file, "w") as f:
        f.write(generated_text)
    
    print(f"Completed probe for {t['id']}. Result saved to {out_file}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Completed probe for triplet_01. Result saved to /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/outputs/IMSB-2025/baseline/runs/IMSB-2025_codegen350M_baseline_20260219_175659/generated/IMSB-2025_codegen350M_triplet_01_20260219_175701.txt
Completed probe for triplet_02. Result saved to /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/outputs/IMSB-2025/baseline/runs/IMSB-2025_codegen350M_baseline_20260219_175659/generated/IMSB-2025_codegen350M_triplet_02_20260219_175701.txt


### Update Notes with Findings

In [7]:
with open(LOG_PATH, "a", encoding="utf-8") as log:
    log.write(f"\n## Experiment Run: {datetime.now().isoformat()}\n")
    log.write(f"- Status: Knowledge localization probes complete.\n")
    log.write(f"- Outputs: {RUN_DIR}\n")
print(f"Notes updated at {LOG_PATH}")

update_manifests(
    run_id=RUN_ID,
    paper_id=PAPER_ID,
    phase=PHASE,
    model=MODEL_TAG,
    timestamp=TIMESTAMP,
    gen_path=GEN_DIR.relative_to(ROOT),
    metrics_path=METRICS_DIR.relative_to(ROOT)
)



Notes updated at /Users/dhrubadatta/Documents/Research/CodeAudit X/Codes/notes/IMSB-2025_notes.md
